In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, f1_score
from sklearn.model_selection import train_test_split
from sklearn import tree
from scipy.stats import wilcoxon
from joblib import Parallel, delayed
from sklearn.datasets import make_classification
import time 

class ARS:
    def __init__(self, X, y, objective='regression', random_state=42):
        self.X = X  # NumPy array
        self.y = y  # NumPy array
        self.objective = objective
        self.random_state = random_state

    def _relevance_score(self, original_median, benchmark, objective):
        if objective == 'classification':
            return (original_median - benchmark) / (1 - benchmark)
        elif objective == 'regression':
            return (benchmark - original_median) / benchmark
        else:
            raise ValueError('Unspecified objective')

    def _acceptable_minimum_distribution(self, base, objective):
        if objective == 'classification':
            return np.round(np.percentile(base, 55), 3) + 0.01
        elif objective == 'regression':
            return np.round(np.percentile(base, 45), 5) - 0.00001
        else:
            raise ValueError('Unspecified objective')

    def _run_iteration(self, iteration, max_depth, stratify=None):
        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(
            self.X, 
            self.y, 
            stratify=stratify, 
            train_size=0.7, 
            random_state=iteration
        )

        # Initialize the original model
        if self.objective == 'regression':
            original_model = tree.DecisionTreeRegressor(
                random_state=iteration, 
                criterion='absolute_error',  # Use 'absolute_error' for MAE
                max_depth=max_depth
            )
            metric = mean_absolute_error
            metric_params = {}
        else:  # 'classification'
            original_model = tree.DecisionTreeClassifier(
                random_state=iteration, 
                criterion='entropy', 
                max_depth=max_depth
            )
            metric = f1_score
            metric_params = {'average': 'micro'}

        # Train the original model
        original_model.fit(X_train, y_train)
        original_pred = original_model.predict(X_test)
        original_score = metric(y_test, original_pred, **metric_params)

        # Initialize the shadow model
        if self.objective == 'classification':
            shadow_model = tree.DecisionTreeClassifier(
                random_state=iteration, 
                criterion='entropy', 
                max_depth=max_depth
            )
        else:
            shadow_model = tree.DecisionTreeRegressor(
                random_state=iteration, 
                criterion='absolute_error', 
                max_depth=max_depth
            )
        
        # Shuffle X_train using a NumPy permutation
        shuffled_indices = np.random.RandomState(iteration).permutation(X_train.shape[0])
        X_train_shuffled = X_train[shuffled_indices]
        shadow_model.fit(X_train_shuffled, y_train)
        shadow_pred = shadow_model.predict(X_test)
        shadow_score = metric(y_test, shadow_pred, **metric_params)

        return original_score, shadow_score

    def calculate_multivariate_relevance_score(self, max_iterations=1000):
        np.random.seed(self.random_state)
        
        # Convert y to the appropriate type
        if self.objective == 'regression':
            y = self.y.astype(float)
            stratify = None
        elif self.objective == 'classification':
            y = self.y.astype(str)
            stratify = y
        else:
            raise ValueError('Specify objective')

        # Check if y has only one unique value
        if np.unique(y).size == 1:
            return [0.0, 0.0, 0.0]

        # Calculate max_depth once (assumes fixed training size)
        sample_size = int(0.7 * self.X.shape[0])
        max_depth = int(np.ceil(np.log2(sample_size)))+1

        # Execute iterations in parallel
        results = Parallel(n_jobs=-1)(
            delayed(self._run_iteration)(i, max_depth, stratify) for i in range(max_iterations)
        )

        # Separate results
        original_scores, shadow_scores = zip(*results)

        # Convert to arrays for efficiency
        A = np.array(original_scores)
        B = np.array(shadow_scores)

        # Calculate metrics
        acceptable_minimum = self._acceptable_minimum_distribution(B, self.objective)
        median_original_value = np.median(A)
        relevance_score = self._relevance_score(median_original_value, acceptable_minimum, self.objective)

        # Wilcoxon test
        if self.objective == 'classification':
            alternative = 'greater'
        else:
            alternative = 'less'

        stat, p_value = wilcoxon(A, B, alternative=alternative)

        # Check conditions for relevance score
        if median_original_value < 0 or relevance_score < 0 or p_value >= 0.05:
            relevance_score = 0.0

        return [relevance_score, median_original_value, acceptable_minimum]



In [2]:
# Function to convert numbers to strings
def to_string(number_list):
    return [str(number) for number in number_list]

# Generate sample data
n_features_ = 30
X, y = make_classification(
    random_state=0,
    n_samples=3000,
    n_features=n_features_,
    n_informative=3,
    n_redundant=0,
    shuffle=False
)

# Convert to NumPy arrays
X = X.astype(float)
y = y.astype(str)

# Create a list of feature names
feature_names = to_string(range(n_features_))

# Function to process each feature and record its processing time
def process_feature(i):
    start_time = time.time()  # Start time
    X_col = X[:, [i]]  # Select a single column
    y_col = y
    ars = ARS(X=X_col, y=y_col, objective='classification', random_state=42)
    score, median, acceptable_min = ars.calculate_multivariate_relevance_score()
    end_time = time.time()  # End time
    elapsed_time = end_time - start_time  # Calculate elapsed time
    return [[feature_names[i]], score, median, acceptable_min, elapsed_time]

# Initialize a list to store the results
results_list = []

# Measure the total processing time
start_time = time.time()

# Execute in parallel for all features
results = Parallel(n_jobs=-1)(
    delayed(process_feature)(i) for i in range(n_features_)
)

end_time = time.time()
total_time =end_time - start_time
print(f"\nTotal Processing Time: {total_time:.2f} seconds")



Total Processing Time: 68.49 seconds


In [3]:
results= pd.DataFrame(results)

In [4]:
results

,0,1,2,3,4
0,[0],0.098791,0.561111,0.513,26.474858
1,[1],0.266161,0.645556,0.517,25.171335
2,[2],0.096084,0.558889,0.512,36.394047
3,[3],0.000000,0.505556,0.512,25.717199
4,[4],0.000000,0.493889,0.512,35.806523
5,[5],0.000000,0.506667,0.512,27.125495
6,[6],0.000000,0.506667,0.512,24.230178
7,[7],0.000000,0.494444,0.512,24.329760
8,[8],0.000000,0.502222,0.512,25.484045
9,[9],0.000000,0.504444,0.511,36.824156
